In [52]:
import numpy as np
import pandas as pd
import zipcodes
import re
import os

In [53]:
# Helper functions
def zip_state(zip_code):
    if pd.isna(zip_code):
        return "unkown"
    zip_str = str(zip_code).zfill(5)
    result = zipcodes.matching(zip_str)
    if result:
        return result[0].get('state', 'unkown')
    return 'unkown'

def clean_zip(zip_code):
    zip_str = str(zip_code).strip().upper()
    if 'X' in zip_str:
        digit_only = ''.join([c for c in zip_str if c.isdigit()])
        if len(digit_only) >= 3:
            return digit_only.ljust(5, '0')
        else:
            return None
    if zip_str.isdigit() and len(zip_str) <= 5:
        return zip_str.zfill(5)
    return None


def group_cats(df, col, threshold=0.01):
    freq = df[col].value_counts(normalize=True)
    rare = freq[freq < threshold].index
    df[col] = df[col].replace(rare, 'Rare')
    return df

def clean_text(text):
    text = str(text).lower().strip()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def export(df, folder_path='../data/', filename='clean_data.csv'):
    full_path = os.path.join(folder_path, filename)
    df.to_csv(full_path, index=False)
    print(f"data exported: {full_path}")


In [54]:
# Primary dataset input and basic information
df = pd.read_csv('../data/data.csv')
df.count()
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407942 entries, 0 to 1407941
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Date received                 1407942 non-null  object 
 1   Product                       1407942 non-null  object 
 2   Sub-product                   1378566 non-null  object 
 3   Issue                         1407942 non-null  object 
 4   Sub-issue                     1272575 non-null  object 
 5   Consumer complaint narrative  1407941 non-null  object 
 6   Company public response       754793 non-null   object 
 7   Company                       1407941 non-null  object 
 8   State                         1402833 non-null  object 
 9   ZIP code                      1407941 non-null  object 
 10  Tags                          155861 non-null   object 
 11  Consumer consent provided?    1407941 non-null  object 
 12  Submitted via               

,Complaint ID
count,1.407941e+06
mean,6.450262e+06
std,2.750105e+06
min,1.290157e+06
25%,3.890939e+06
50%,6.536953e+06
75%,8.778808e+06
max,1.167791e+07


In [55]:
# Dropping exact dupes and standarization of column names
df.drop_duplicates(inplace=True)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [56]:
# Date handeling
df['date_received'] = pd.to_datetime(df['date_received'], errors='coerce')
df['date_sent_to_company'] = pd.to_datetime(df['date_sent_to_company'], errors='coerce')

/var/folders/zg/_j5hfs9d12d155g0l1sy26n00000gn/T/ipykernel_26079/4029791801.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_received'] = pd.to_datetime(df['date_received'], errors='coerce')
/var/folders/zg/_j5hfs9d12d155g0l1sy26n00000gn/T/ipykernel_26079/4029791801.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_sent_to_company'] = pd.to_datetime(df['date_sent_to_company'], errors='coerce')


In [57]:
# Drop unnessary columns
dropping_cols = ['submitted_via', 'consumer_consent_provided?', 'complaint_id', 'timely_response?', 'consumer_disputed?']
df.drop(columns=dropping_cols, inplace=True)

In [58]:
# zip_code malformation handeling
df['zip_code'] = df['zip_code'].apply(clean_zip)
df['zip_code'] = df['zip_code'].fillna('00000')
df.info()
df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407942 entries, 0 to 1407941
Data columns (total 13 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   date_received                 1407942 non-null  datetime64[ns]
 1   product                       1407942 non-null  object        
 2   sub-product                   1378566 non-null  object        
 3   issue                         1407942 non-null  object        
 4   sub-issue                     1272575 non-null  object        
 5   consumer_complaint_narrative  1407941 non-null  object        
 6   company_public_response       754793 non-null   object        
 7   company                       1407941 non-null  object        
 8   state                         1402833 non-null  object        
 9   zip_code                      1407942 non-null  object        
 10  tags                          155861 non-null   object        
 11

,date_received,product,sub-product,issue,sub-issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,date_sent_to_company,company_response_to_consumer
0,2023-11-22,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,This is my multiple endeavor to tell you that ...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,92335,NaN,2023-11-22,Closed with non-monetary relief
1,2019-05-21,Credit card or prepaid card,General-purpose credit card or charge card,"Advertising and marketing, including promotion...",Didn't receive advertised or promotional terms,"On approximately XX/XX/19, I took advantage of...",Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",IL,60500,NaN,2019-05-21,Closed with explanation
2,2018-12-26,"Money transfer, virtual currency, or money ser...",International money transfer,Other transaction problem,NaN,The money transfer got cancelled last minute f...,NaN,"WESTERN UNION COMPANY, THE",CT,06611,NaN,2018-12-26,Closed with explanation
3,2017-05-22,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,"On XXXX XXXX , XXXX , I contacted Monterey...",Company believes it acted appropriately as aut...,Monterey Financial Services LLC,CA,95776,NaN,2017-05-22,Closed with explanation
4,2023-07-03,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,The credit bureau stated my account was proper...,NaN,"EQUIFAX, INC.",GA,30815,NaN,2023-07-03,Closed with explanation
5,2024-02-15,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,I am a victim of identity-theft. I am writing ...,NaN,"EQUIFAX, INC.",NJ,07600,NaN,2024-02-15,Closed with explanation
6,2024-11-08,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account information incorrect,I have submitted multiple disputes regarding t...,NaN,"EQUIFAX, INC.",NY,11756,NaN,2024-11-08,Closed with non-monetary relief
7,2022-10-28,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,Credit Reporting Agency Transunion continues t...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,34219,NaN,2022-10-28,Closed with non-monetary relief
8,2022-09-20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,I have disputed the referenced accounts to the...,NaN,"EQUIFAX, INC.",NY,10462,NaN,2022-09-20,Closed with explanation
9,2022-09-10,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Difficulty submitting a dispute or getting inf...,I have tried to send my disputes to the major ...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,IN,46300,NaN,2022-09-10,Closed with non-monetary relief


In [59]:
# Null handeling
# tags
df['tags'] = df['tags'].fillna('none')
# company_public_response
df['company_public_response'] = df['company_public_response'].fillna('no response')
# sub-issue
df['sub-issue'] = df['sub-issue'].fillna('none')
# sub-product
df['sub-product'] = df['sub-product'].fillna('none')
# state
df['state'] = df.apply(lambda row: row['state'] if pd.notna(row['state']) else zip_state(row['zip_code']), axis = 1)
# company_response_to_consumer
df['company_response_to_consumer'] = df['company_response_to_consumer'].fillna('no response')
# consumer_complaint_narrative
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].fillna('')
# company
df['company'] = df['company'].fillna('unknown')
# date_sent_to_company
df['date_sent_to_company'] = df['date_sent_to_company'].fillna(df['date_received'])
df['date_sent_to_company'] = df['date_sent_to_company'].fillna(pd.Timestamp('2000-01-01'))

df.isnull().mean().sort_values(ascending=False)


date_received                   0.0
product                         0.0
sub-product                     0.0
issue                           0.0
sub-issue                       0.0
consumer_complaint_narrative    0.0
company_public_response         0.0
company                         0.0
state                           0.0
zip_code                        0.0
tags                            0.0
date_sent_to_company            0.0
company_response_to_consumer    0.0
dtype: float64

In [60]:
# Derived features for later analysis
df['process_days'] = (df['date_sent_to_company'] - df['date_received']).dt.days
df['year_received'] = (df['date_received']).dt.year
df['month_received'] = (df['date_received']).dt.month

# Double check nulls
df.isnull().mean().sort_values(ascending=False)


date_received                   0.0
product                         0.0
sub-product                     0.0
issue                           0.0
sub-issue                       0.0
consumer_complaint_narrative    0.0
company_public_response         0.0
company                         0.0
state                           0.0
zip_code                        0.0
tags                            0.0
date_sent_to_company            0.0
company_response_to_consumer    0.0
process_days                    0.0
year_received                   0.0
month_received                  0.0
dtype: float64

In [61]:
# Categorical variables
cat_columns = [
    'product',
    'sub-product',
    'issue',
    'sub-issue',
    'company',
    'state',
    'tags',
    'company_response_to_consumer',
    'month_received',
    'year_received'
]

for col in cat_columns:
    df[col] = df[col].astype('category')

In [62]:
# Grouping
df = group_cats(df, 'company')
df = group_cats(df, 'sub-issue')

/var/folders/zg/_j5hfs9d12d155g0l1sy26n00000gn/T/ipykernel_26079/1182789991.py:27: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[col] = df[col].replace(rare, 'Rare')
/var/folders/zg/_j5hfs9d12d155g0l1sy26n00000gn/T/ipykernel_26079/1182789991.py:27: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[col] = df[col].replace(rare, 'Rare')


In [63]:
# Text prep
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].apply(clean_text)
df['has_narrative'] = df['consumer_complaint_narrative'].apply(lambda x: bool(x.strip()))

In [64]:
# Export
export(df)

data exported: ../data/clean_data.csv
